In [1]:
# Calculate the Gini index for a split dataset
def gini_index(groups, class_values):
    # two elements in groups, each a list of rows of the data points belonging to that group
	gini = 0.0
	for class_value in class_values:
		for group in groups:
			size = len(group)
			if size == 0:
				continue
			proportion = [row[-1] for row in group].count(class_value) / float(size)
            # the class value label is the last element of each row of the dataset
            # counts the occurrences of a class value in a group
			gini += (proportion * (1.0 - proportion))
	return gini

In [2]:
# test Gini values
print(gini_index([[[1, 1], [1, 0]], [[1, 1], [1, 0]]], [0, 1]))
print(gini_index([[[1, 0], [1, 0]], [[1, 1], [1, 1]]], [0, 1]))
# the first index is redundant (used to match with the actual data which contains the class label as the 
# last element)

1.0
0.0


In [3]:
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
    # index is the index of the attribute being tested in each vector
    # value is the threshold value
	left, right = list(), list()
	for row in dataset:
		if row[index] < value:
			left.append(row)   # increasing towards the right
		else:
			right.append(row)
	return left, right

In [4]:
# Select the best split point for a dataset
def get_split(dataset):
	class_values = list(set(row[-1] for row in dataset))
	b_index, b_value, b_score, b_groups = 999, 999, 999, None

	for index in range(len(dataset[0])-1):
		for row in dataset:
			groups = test_split(index, row[index], dataset)
			gini = gini_index(groups, class_values)
			if gini < b_score:
				b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    # loop over each row of the dataset to find the split value which gives least gini index corresponding to
    # a selected attribute
    # then loop over all attributes
	return {'index':b_index, 'value':b_value, 'groups':b_groups}

In [5]:
# example for testing
dataset = [[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]]
split = get_split(dataset)
print('Split: [X%d < %.3f]' % ((split['index']+1), split['value']))

Split: [X1 < 6.642]


In [6]:
# Create a terminal node value
# selects the mode of the group as the class value of a group of rows
def to_terminal(group):
	outcomes = [row[-1] for row in group]
	return max(set(outcomes), key=outcomes.count)

In [7]:
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
    # min_size is the minimum number of data points associated with a node such that it is a decision node
    
	left, right = node['groups']
	del(node['groups'])
    # deleting redundant data
    
	# check for a no split
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return
    
	# check for max depth
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
    
	# process left child
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
        
	else:
		node['left'] = get_split(left)
		split(node['left'], max_depth, min_size, depth+1)
        
	# process right child
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
        
	else:
		node['right'] = get_split(right)
		split(node['right'], max_depth, min_size, depth+1)

In [8]:
# Build a decision tree
def build_tree(train, max_depth, min_size):
	root = get_split(train)
	split(root, max_depth, min_size, 1)
	return root

In [9]:
# Make a prediction with a decision tree
def predict(node, row):
	if row[node['index']] < node['value']:
		if isinstance(node['left'], dict): # if left child node exists
			return predict(node['left'], row)
		else:
			return node['left']
	else:
		if isinstance(node['right'], dict):
			return predict(node['right'], row)
		else:
			return node['right']

In [10]:
from random import seed
from random import randrange
from csv import reader

# Load a CSV file
def load_csv(filename):
	file = open(filename, "rb")
	lines = reader(file)
	dataset = list(lines)
	return dataset

In [11]:
# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

In [12]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
            # generates a random number in range (0, len(dataset))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split
# dataset_split is a list with n_folds elements, each a list of datapoints and of size=fold_size 

In [13]:
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

In [14]:
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores

In [15]:
# Classification and Regression Tree Algorithm
def decision_tree(train, test, max_depth, min_size):
	tree = build_tree(train, max_depth, min_size)
	predictions = list()
	for row in test:
		prediction = predict(tree, row)
		predictions.append(prediction)
	return(predictions)

In [16]:
# Test CART on Bank Note dataset
seed(1)

# load and prepare data
filename = 'data_banknote_authentication.csv'
dataset = load_csv(filename)

# convert string attributes to integers
for i in range(len(dataset[0])):
	str_column_to_float(dataset, i)
    
# evaluate algorithm
n_folds = 5
max_depth = 5
min_size = 10
scores = evaluate_algorithm(dataset, decision_tree, n_folds, max_depth, min_size)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [82.84671532846716, 83.57664233576642, 85.76642335766424, 80.65693430656934, 81.75182481751825]
Mean Accuracy: 82.920%


possible extensions :  
1. algo tuning : parameter values  
2. cross entropy cost function  
3. tree pruning (advanced)